In [24]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import scipy as sp

In [ ]:
vectorizer = CountVectorizer(min_df=1)

In [3]:
content = ["How to format my hard disk", " Hard disk format problems"]
X = vectorizer.fit_transform(content)
vectorizer.get_feature_names()

['disk', 'format', 'hard', 'how', 'my', 'problems', 'to']

In [10]:
X.toarray()

array([[1, 1, 1, 1, 1, 0, 1],
       [1, 1, 1, 0, 0, 1, 0]], dtype=int64)

In [11]:
posts = ["This is a toy post about machine learning. Actually, it contains not much interesting stuff.", 
         "Imaging databases can get huge.",
         "Most imaging databases save images permanently.",
         "Imaging databases store images.",
         "Imaging databases store images. Imaging databases store images.Imaging databases store images."]

In [12]:
X_train = vectorizer.fit_transform(posts)

In [13]:
num_samples, num_features = X_train.shape

In [14]:
X_train.shape

(5, 24)

In [21]:
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])

In [23]:
print(new_post_vec)
print(new_post_vec.toarray())

  (0, 4)	1
  (0, 8)	1
[[0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [25]:
def dist_raw(v1,v2):
    delta = v1-v2
    return sp.linalg.norm(delta.array())
# Distance Coefficients between Two Lists or Sets in The Python Papers Source Codes

In [26]:
num_samples

5

In [29]:
for i,post in enumerate(X_train):
    print(i,post)

0   (0, 22)	1
  (0, 10)	1
  (0, 23)	1
  (0, 18)	1
  (0, 0)	1
  (0, 13)	1
  (0, 12)	1
  (0, 1)	1
  (0, 11)	1
  (0, 3)	1
  (0, 16)	1
  (0, 15)	1
  (0, 9)	1
  (0, 21)	1
1   (0, 8)	1
  (0, 4)	1
  (0, 2)	1
  (0, 5)	1
  (0, 6)	1
2   (0, 8)	1
  (0, 4)	1
  (0, 14)	1
  (0, 19)	1
  (0, 7)	1
  (0, 17)	1
3   (0, 8)	1
  (0, 4)	1
  (0, 7)	1
  (0, 20)	1
4   (0, 8)	3
  (0, 4)	3
  (0, 7)	3
  (0, 20)	3
